In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import json

from dotenv import load_dotenv
load_dotenv()

import ollama

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import InjectedState

from scripts.file_tools import (
    DeepAgentState,
    ls,
    read_file,
    write_file,
    cleanup_files,
)
from scripts.prompts import (
    ORCHESTRATOR_PROMPT,
    RESEARCHER_PROMPT,
    EDITOR_PROMPT,
)

In [ ]:
# -------------------------
# Web Search Tool
# -------------------------

@tool
def web_search(query: str) -> str:
    """
    Perform a live web search using Ollama Cloud Web Search API.

    Input:
        query: search query string

    Output:
        JSON string of top results (max_results=2).
    """
    response = ollama.web_search(query, max_results=2)
    response = response.results

    return response


In [ ]:
# result = web_search.invoke({"query": "Latest global news"})

In [ ]:
# -------------------------
# LLM
# -------------------------

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

In [ ]:
# -------------------------
# Worker Agents: Researcher & Editor
# -------------------------

researcher_tools = [ls, write_file, read_file, web_search]
researcher_agent = create_agent(
    model=llm,
    tools=researcher_tools,
    system_prompt=RESEARCHER_PROMPT,
    state_schema=DeepAgentState,
)

editor_tools = [ls, read_file, write_file, cleanup_files]
editor_agent = create_agent(
    model=llm,
    tools=editor_tools,
    system_prompt=EDITOR_PROMPT,
    state_schema=DeepAgentState,
)


In [ ]:
# -------------------------
# Orchestrator Tools (call other agents)
# -------------------------
from typing import Annotated
from langgraph.prebuilt import InjectedState
from langchain_core.tools import InjectedToolCallId
from langgraph.types import Command

@tool
def run_researcher(
    theme_id: int,
    thematic_question: str,
    state: Annotated[DeepAgentState, InjectedState],
    max_retries: int = 2
) -> str:
    """
    Run a single Research agent for ONE thematic question.

    Args:
        theme_id: The theme number (1, 2, 3, etc.)
        thematic_question: The specific thematic question to research
        state: Injected agent state providing user_id/thread_id
        max_retries: Number of retry attempts if researcher fails

    The Researcher will:
    - Receive ONE thematic question as input
    - Break it into 2-4 focused search queries
    - Use web_search to gather information
    - Write files to researcher/ folder with hash-based names:
        * researcher/<hash>_theme.md (research findings)
        * researcher/<hash>_sources.txt (raw sources)

    Returns a short status string for the orchestrator.
    """
    from scripts.file_tools import generate_hash
    
    # Generate hash for unique file naming
    file_hash = generate_hash(f"{theme_id}_{thematic_question}")
    
    # Build sub-state with theme-specific context
    sub_state: DeepAgentState = {
        "messages": state["messages"] + [
            AIMessage(content=f"[THEME {theme_id}] Research this question: {thematic_question}\n\n"
                             f"File hash: {file_hash}\n"
                             f"Save your findings to: researcher/{file_hash}_theme.md\n"
                             f"Save your sources to: researcher/{file_hash}_sources.txt")
        ],
        "user_id": state.get("user_id"),
        "thread_id": state.get("thread_id"),
    }
    
    # Execute researcher with retry logic
    for attempt in range(max_retries + 1):
        try:
            researcher_agent.invoke(sub_state)
            return f"✓ Theme {theme_id} research completed (hash: {file_hash})"
        except Exception as e:
            if attempt < max_retries:
                print(f"⚠ Theme {theme_id} failed (attempt {attempt + 1}/{max_retries + 1}), retrying...")
                continue
            else:
                return f"✗ Theme {theme_id} failed after {max_retries + 1} attempts: {str(e)}"
    
    return f"✓ Theme {theme_id} research completed (hash: {file_hash})"


In [ ]:
@tool
def write_research_plan(
    thematic_questions: list[str],
    state: Annotated[DeepAgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId],
) -> Command:
    """
    Write the high-level research plan with major thematic questions.

    Args:
        thematic_questions: List of 3-5 major thematic questions that break down
                           the user's query. These guide the Researcher agent.
        state: Injected agent state providing user_id/thread_id.
        tool_call_id: Tool call ID for attaching a ToolMessage.

    The research_plan.md will be read by the Researcher to guide tactical research.

    Returns:
        Command with a ToolMessage confirming the plan was written.
    """
    from scripts.file_tools import _disk_path
    from langchain_core.messages import ToolMessage
    from langgraph.types import Command
    
    # Build content for research_plan.md
    content = "# Research Plan\n\n"
    content += "## User Query\n"
    # Extract user query from messages
    user_msg = [m for m in state["messages"] if hasattr(m, 'content')]
    if user_msg:
        content += f"{user_msg[-1].content}\n\n"
    
    content += "## Thematic Questions\n\n"
    for i, question in enumerate(thematic_questions, 1):
        content += f"{i}. {question}\n"
    
    # Write to disk
    path = _disk_path(state, "research_plan.md")
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    
    msg = f"[RESEARCH PLAN WRITTEN] research_plan.md with {len(thematic_questions)} thematic questions -> {path}"
    return Command(
        update={
            "messages": [ToolMessage(msg, tool_call_id=tool_call_id)]
        }
    )


@tool
def run_editor(
    state: Annotated[DeepAgentState, InjectedState]
) -> str:
    """
    Run the Editor agent for this user/thread.

    The Editor will:
    - read research_plan.md, theme files, sources.txt
    - write the final answer to report.md

    Returns a short status string for the orchestrator.
    """
    sub_state: DeepAgentState = {
        "messages": state["messages"],
        "user_id": state.get("user_id"),
        "thread_id": state.get("thread_id"),
    }
    editor_agent.invoke(sub_state)
    return "Editor completed. Final report is written to report.md."

In [ ]:
# -------------------------
# Orchestrator Agent
# -------------------------

orchestrator_tools = [
    write_research_plan,  # strategic planning
    run_researcher,       # tactical research
    run_editor,           # synthesis
    cleanup_files,        # resetting workspace
]

orchestrator_agent = create_agent(
    model=llm,
    tools=orchestrator_tools,
    system_prompt=ORCHESTRATOR_PROMPT,
    state_schema=DeepAgentState,
)


### Example usage


In [ ]:
from langchain.messages import HumanMessage
state = {
        "messages": [HumanMessage(content="Tell me a joke about computers.")],
        "user_id": "user_123",
        "thread_id": "thread_mcp_001",
    }

result = orchestrator_agent.invoke(state)
result

In [ ]:
from langchain.messages import HumanMessage
state = {
        "messages": [HumanMessage(content="Do a detailed, well-structured analysis of MCP including history")],
        "user_id": "user_123",
        "thread_id": "thread_mcp_001",
    }

result = orchestrator_agent.invoke(state)

In [ ]:
result

In [ ]:
from langchain.messages import HumanMessage
state = {
        "messages": [HumanMessage(content="Do a detailed, well-structured comparison of MCP and API.")],
        "user_id": "user_123",
        "thread_id": "thread_mcp_001",
    }

result = orchestrator_agent.invoke(state)

In [ ]:
result